# 大規模データ解析入門 (3)
シミュレーションで観察データを評価する

## Contents
### Introduction
- [前回の復習](#0.1)
- [今回のサンプルデータ](#0.2)
- [今回の実習](#0.3)

--- 

### Practice
1. [シミュレーションでSNP-indexを1個作る](#1.1)
1. [シミュレーションでSNP-indexを10個作る](#1.2)
1. [シミュレーションでSNP-indexを10000個作る](#1.3)
1. [観察データは帰無仮説のもとでよく得られる値か？](#1.4)
1. [原因遺伝子領域を特定する](#1.5)

## 前回の復習<a name="0.1"></a>

前回の[実習内容](../07_large_data_analysis/02_large_data_analysis_ja.ipynb)は次のとおりです。
- タブ区切りテキストファイルの読み込み、pandasデータフレームにする
    - ```python
        import pandas as pd
        df = pd.read_csv(ファイル名, sep='区切り文字', header=ヘッダーの行番号, names=[列名のリスト])
        ```
- 列データ、行データ、1つのセルデータにアクセスする
    - `df[列名]` => 列データの取り出し
    - `df.loc[行名,列名]` => 行データや列データの取り出し
    - `df.iloc[行番号,列番号]` => 行データや列データの取り出し
- 列間で数値計算し、新しい列をデータフレームに追加する
    - `df[C列] = df[A列] + df[B列]`
- 条件を指定して、一部データのみを抽出する
    - `df[ (条件式) ]`
    - AND条件(`&`)、OR条件(`|`)
    - 例) `df[ (df['snp_index'] >= 0.7) & (df['snp_index'] < 0.9) ]`
- データをグラフ（散布図）にする
    - 「Matplotlib」というライブラリを使って作図。
    - 「レイヤー」と呼ばれる透明なシート一枚一枚にグラフ本体やタイトル・軸ラベルの文字が描かれている。 
    - それらレイヤーが複数重なることで一つのグラフが出来上がっている。
    - ```python
        %matplotlib inline
        import matplotlib.pyplot as plt  # matplotlibライブラリの準備
        
        fig = plt.figure()  # 作図フィールドの設定
        plt.scatter([x軸データのリスト], [y軸データのリスト])  # 散布図
        plt.title('グラフタイトル')  # グラフタイトル
        plt.xlabel('x軸ラベル')      # x軸ラベル
        plt.ylabel('y軸ラベル')      # y軸ラベル
        ```
- それらを使って、データ解析（Sliding window解析）をする
    - ゲノム上にいくつか区間を設けて、それぞれの区間において平均値や合計値、個数を算出する解析
    - それを折れ線グラフにすれば、ゲノム上で調べている値がどのように増減するかを視覚化できる

## 今回のサンプルデータ<a name="0.2"></a>

前回までと同様に、MutMap(Abe et al., 2012)のテーブルデータを使います。  

実習前に下のセルを実行してください。

ファイルのダウンロードと、そのファイルを`df`という名前のpandasデータフレームに入れています。  
また、SNP-indexを計算して、`snp_index`列として追加しています。

In [ ]:
"""
＊重要＊
最初にこのセルを実行してください。
サンプルファイルをダウンロードします。
"""
!wget -q https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_bulk.txt -O mutmap_bulk.txt
    
#--- pandasを読み込む  ---
import pandas as pd

#--- 読み込みファイル名 ---
dataset = 'mutmap_bulk.txt'       

#--- ファイルを読み込む ---
# header=-1（列名行なし）を指定した場合、列名として通し番号が付けられます。
# names=[リスト]とすることで、通し番号の代わりに列名を指定することが可能です。
df = pd.read_csv(dataset, sep="\t", header=-1, names=['chr', 'pos', 'ref_nucl', 'alt_nucl', 'ref_N', 'alt_N'])

#--- SNP-index計算 ---
df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])

#--- 表示(10行分のデータのみ) ---
df.head(10)

## 今回の実習<a name="0.3"></a>

突然変異形質（葉が白くなる形質）の原因遺伝子領域の同定をより客観的におこなうために、シミュレーションにより大量のSNP-indexを得て、それらを使い観察データ（実データ）の評価をおこないます。

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/08/simulation01_ja.jpg?raw=true" alt="simulation"></div>

---
## Practice
Practice 1-4では、「0番目」のデータ（`chr10	51406	G	A	6	3	0.333333`）のみを取り扱います。

In [ ]:
# データ抽出
onedata = df.iloc[0,:]

# SNP-indexを「mysnpindex」という変数に入れている
# （STEP 4で使います）
mysnpindex = onedata['snp_index']

# 表示
onedata

サンプルデータでは、chromosome 10の51406番目の塩基上には、9個の塩基がのっています。  
そのうち、6個がオリジナル塩基`G`で、３個が突然変異により生じた塩基`A`です。

<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/08/simulation02_ja.jpg?raw=true" alt="simulation"></div>

__問: 9個の塩基のうち、`G`が6個、`A`が3個得られる確率はどのぐらいか？__  

ここでは、次の条件下で、10,000個のSNP-indexをシミュレーションで発生させて、「９個の塩基のうち、`G`が6個、`A`が3個得られる確率」を調べてみましょう。

__条件: 2つの塩基（`G`と`A`）がそれぞれ50%の確率で得られる。__

実際のMutMap法（Abe et al., 2012）では、もう少し多くシミュレーションをおこなっています（注1）。  

ここではかなり単純化させたシミュレーションをおこなうため、2つの塩基それぞれが選ばれる確率を「50%」としています。  
これは、MutMap法において形質にかかわる遺伝子から遠く離れた塩基座位では、ほぼ50%の確率で対立する2つの塩基が後代（F2世代）に伝わっていると期待されるためです。

したがって、このシミュレーションで発生させるSNP-indexは、「形質と無関係な塩基座位で得られるSNP-index」です（帰無仮説）。  

実際に観察されたSNP-index（実データ）が、この帰無仮説の下ではめったに発生しないSNP-index（多くの場合、分布の端の5%以下でしか得られないSNP-index）であった場合、帰無仮説を棄却し「形質と無関係な塩基座位」ではないと判定します。

（注1） MutMap法では、「F2集団作成」と「各F2個体の遺伝子型作成」、「突然変異形質20個体の選抜」、「20個体中の塩基頻度」、「その塩基頻度を使ったSNP-indexの発生（今回の実習では塩基頻度を50%に固定）」といった一連のシミュレーションを10,000回おこなっています。

### 1. シミュレーションでSNP-indexを1個作る<a name="1.1"></a>

まずは1個のSNP-indexをシミュレーションで発生させてみましょう。

In [ ]:
import numpy as np  # NumPyライブラリ

# 何個の塩基を発生させるか（=合計塩基数）
total_allele_num = 9

# 二項分布にしたがう場合、突然変異塩基が何個選ばれるかシミュレーションする
simu_mut_num = np.random.binomial(n=total_allele_num, p=0.5)
print(simu_mut_num)

# シミュレーションで得られた突然変異塩基の個数から、SNP-indexを求める。
# 計算式: 突然変異塩基数　/ 合計塩基数
#simu_snp_index = simu_mut_num / total_allele_num
#print(simu_snp_index)

### 2. シミュレーションでSNP-indexを10個作る<a name="1.2"></a>
次に10個のSNP-indexを発生させてみましょう。

In [ ]:
"""
方法1: for文を使う （Bad method）
"""

import numpy as np  # NumPyライブラリ

# 何個の塩基を発生させるか（=合計塩基数）
total_allele_num = 9

# 発生させた値を入れるリスト
simu_mut_nums = []
simu_snp_indexes = []

# 10回シミュレーションする
for i in range(10):
    # 二項分布にしたがう場合、突然変異塩基が何個選ばれるかシミュレーションする
    simu_mut_num = np.random.binomial(n=total_allele_num, p=0.5)

    # シミュレーションで得られた突然変異塩基の個数から、SNP-indexを求める。
    # 計算式: 突然変異塩基数　/ 合計塩基数
    simu_snp_index = simu_mut_num / total_allele_num
    
    #　リストに追加
    simu_mut_nums.append(simu_mut_num)
    simu_snp_indexes.append(simu_snp_index)

# 表示
print(simu_mut_nums)
print(simu_snp_indexes)

In [ ]:
"""
方法2: numpy.random.binomial()の引数sizeを使う （Good method）
"""

import numpy as np  # NumPyライブラリ

# 何個の塩基を発生させるか（=合計塩基数）
total_allele_num = 9

# 二項分布にしたがう場合、突然変異塩基が何個選ばれるかシミュレーションする
# そのシミュレーションを10回おこなう
simu_mut_num = np.random.binomial(n=total_allele_num, p=0.5, size=10)
print(simu_mut_num)

# シミュレーションで得られた突然変異塩基の個数から、SNP-indexを求める。
# 計算式: 突然変異塩基数　/ 合計塩基数
simu_snp_index = simu_mut_num / total_allele_num
print(simu_snp_index)

#### 補足: リスト型とNumPy配列型
NumPyの配列（リストみたいなデータ型）は、ある値をすべての要素に対して掛けたり、割ったりすることが可能です。
（これは「ブロードキャスト」と言われる機能です。詳しく知りたい方は調べてみてください）

これまで習ってきた「リスト型」にはブロードキャスト機能はありません。

実際の大規模データを扱う際、実行速度が非常に重要になってきます。

データ型は、リスト型 <=> NumPy配列型 と相互に変換可能なので、
データの追加・削除時には「リスト型」、計算時には「NumPy配列型」、
といった使い分けが実行速度をあげるポイントになってきます。

＊ データの追加・削除は「リスト型」のほうが得意（速い）です。

In [ ]:
# データ型の調べ方とその違い

import numpy as np

a = [1,2,3,4]
b = np.array([1,2,3,4])

# データ型はtype()で調べることが可能
print('a:', a, '  data-type: ', type(a))
print('b:', b, '  data-type: ', type(b))

print('---')
print(a + a) # joining two list
print(b + b) # adding each values

print('---')
print(a * 5) # duplicating the list and joining them
print(b * 5) # multiplicating to each values

### 3. シミュレーションでSNP-indexを10,000個作る<a name="1.3"></a>

以下のプログラムをSNP-index 10,000個用に変更してください。  
（初期状態は「2. シミュレーションでSNP-indexを10個作る」の方法2のプログラムです）

In [ ]:
"""
!!! 実習 !!!
10,000個のSNP-indexを発生させるプログラムにするために、
以下のプログラムのどこかを変更してください。
"""

import numpy as np  # NumPyライブラリ

# 何個の塩基を発生させるか（=合計塩基数）
total_allele_num = 9

# 二項分布にしたがう場合、突然変異塩基が何個選ばれるかシミュレーションする
# 10回分のシミュレーションをおこなう
simu_mut_num = np.random.binomial(n=total_allele_num, p=0.5, size=10)
print(simu_mut_num)

# シミュレーションで得られた突然変異塩基の個数から、SNP-indexを求める。
# 計算式: 突然変異塩基数　/ 合計塩基数
simu_snp_index = simu_mut_num / total_allele_num
print(simu_snp_index)

発生させたSNP-indexの分布（ヒストグラム）を描いてみましょう。  
上のセルで正しくプログラムを変更できていれば、変数`simu_snp_index`に10,000個のSNP-indexが入っています。  

In [ ]:
# matplotlibを準備
import matplotlib.pyplot as plt
%matplotlib inline

# グラフフィールドの設定
fig = plt.figure(figsize=[9,4])    

# ヒストグラムを描く
plt.hist(simu_snp_index, bins=10, range=[0.0,1.0], rwidth=0.8)
        # binsで「表示するbarの数」を指定している
        # rangeで最小値と最大値を指定している
        # rwidthで「表示するbarの横幅」を指定している

# グラフタイトル、x軸ラベル、y軸ラベル
plt.title('Histgram of 10,000 simulated SNP-index', fontsize=16)
plt.xlabel('SNP-index', fontsize=16)  
plt.ylabel('Counts', fontsize=16)

実データ「0番目」のSNP-index=0.333を含むbarのカウント数とその割合は？

In [ ]:
# 配列simu_snp_indexから、0.3以上0.4未満の値のみを抽出
bar_vals = simu_snp_index[ (simu_snp_index >= 0.3) & (simu_snp_index < 0.4)]
print(bar_vals)

# 抽出した値の個数とその割合を調べる
bar_counts = len(bar_vals)
print(bar_counts)
print(bar_counts / 10000)

今回の場合、9個塩基のうち、突然変異塩基`A`が何個選ばれるかをシミュレーションで調べたため、0.3以上0.4未満のbarには0.333...の値のみしか含まれません。  

したがって、__「９個の塩基のうち、Gが6個、Aが3個得られる確率」__は上のセルで得られた割合に等しいです。

### 補足: 二項定理の方程式から確率を計算する
__「９個の塩基のうち、Gが6個、Aが3個得られる確率」__は、二項定理の方程式を解いて確率を調べることも可能です。

$$
P[X=k] = {}_n\mathrm{C}_k p^k (1-p)^{n-k} = \dfrac {n!}{k! (n-k)!} p^k (1-p)^{n-k} 
$$


In [ ]:
import math

P = math.factorial(9) / (math.factorial(3) * math.factorial(6)) * 0.5**3 * 0.5 **6

P

### 4. 観察データは帰無仮説のもとでよく得られる値か？<a name="1.4"></a>

以下、今回の実習の序文を再喝しています。

```text
このシミュレーションで発生させるSNP-indexは、「形質と無関係な塩基座位で得られるSNP-index」です（帰無仮説）。

実際に観察されたSNP-index（実データ）が、この帰無仮説の下ではめったに発生しないSNP-index（多くの場合、分布の端の5%以下でしか得られないSNP-index）であった場合、帰無仮説を棄却し「形質と無関係な塩基座位」ではないと判定します。
```

Practice ３の解析で、実データ`SNP-index=0.333`が約16%で得られるような値であるという結果が得られていますが、ここでは「分布の端5%以下」に含まれるかどうかといった視点で、実データを検証してみましょう。

_分布の端は両側にありますが、ここでは右端（1.0側）のみを「分布の端」として扱います。_

#### 解析手順
1. 10,000回シミュレーションの値を大きい順に並べる。
1. 分布の端5%の値、すなわち、上位95%の位置の値を得る。
1. 実データとその値を比較する。
1. その値より大きければ、実データは「分布の端5%以下」の値である（帰無仮説を棄却）。  
    そうでなければ、分布の95%内に含まれる値である（帰無仮説を採用）。
    
<div style="margin-bottom: 5px;"><img src="https://github.com/CropEvol/lecture/blob/master/images/08/simulation03_ja.jpg?raw=true" alt="simulation"></div>

いま、変数`simu_snp_index`に10,000個のSNP-index、変数`mysnpindex`に実データのSNP-indexが入っています。  

In [ ]:
import numpy as np

# 実データ（観察されたSNP-index）を表示
print('Observed SNP-index: ', mysnpindex)

# 1) 10,000回シミュレーションの値を大きい順に並べる。
# 2) 上位95%の位置の値を得る。
# numpy.percentile()でこれらを簡単におこなうことが可能です。
get_snp_index  = np.percentile(simu_snp_index, 95, interpolation='lower')

print('SNP-index of threshold: ', get_snp_index) #表示

# 3) 実データと上位95%の位置の値を比較する
if get_snp_index < mysnpindex:
    # 4.1) 大きい場合、帰無仮説を棄却
    print('Rejecting the null hypothesis') 
else:
    # 4.2) 小さい場合、帰無仮説を採用
    print('Adopting the null hypothesis')

### 5. 原因遺伝子領域を特定する<a name="1.5"></a>

これまでは一つの塩基座位のSNP-indexを検証してきました。

ここからは、染色体上の全ての塩基座位についてSNP-indexのシミュレーションをおこないます。  
シミュレーション用の自作関数`snp_index_simulation`を用意しています。

In [ ]:
import numpy as np

#=== 自作関数: SNP-indexシミュレーション ===
def snp_index_simulation(df, ref_colname, alt_colname, trials=10000):
    """
    SNP-indexシミュレーション関数
    
    引数:
    - df : データフレーム（必須）
    - ref_colname: オリジナル塩基数の列名（必須）
    - alt_colname: 突然変異塩基数の列名（必須）
    - trials : 一つの塩基座位において発生させるSNP-index数（初期値:10000）
    
    戻り値: データフレーム
    * シミュレーションで発生させたSNP-indexうち、上位95%にあたる値を新しい列として追加
    """
    
    # 1塩基座シミュレーション関数
    def one_locus_simulation(allele_num):
        # 塩基の選抜
        simu_allele_nums  = np.random.binomial(n=allele_num, p=0.5, size=trials)
        # SNP-indexを計算
        simu_snp_index = simu_allele_nums / allele_num
        # 上位95%の位置にある値（戻り値）
        get_val  = np.percentile(simu_snp_index, 95, interpolation='lower')
        return get_val
    
    # 合計塩基数
    total = df[ref_colname] + df[alt_colname]
    
    # 各塩基座でシミュレーションをおこなう
    # total.map()により、totalに入っているすべての値に対して、
    # 1塩基座シミュレーション関数one_locus_simulation()を実行している
    df['simu95'] = total.map(one_locus_simulation)
    
    # 戻り値
    return df


#=== メイン・プログラム ===
df2 = snp_index_simulation(df, ref_colname='ref_N', alt_colname='alt_N', trials=10000)

# 表示
#df2.head(10) # 最初の10データのみ
df2

実データとシミュレーションデータのSliding window解析をおこない、グラフとして表示してみましょう。  
こちらもSliding Window解析用の自作関数`sliding_window`を用意しています。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#=== 自作関数:  Sliding window解析 ===
def sliding_window(df, pos_colname, val_colname, winsize=1000000, stepsize=200000, chromsize=None):
    """
    Sliding window解析関数
    
    引数:
    - df : データフレーム（必須）
    - pos_colname: 染色体上位置の列名（必須） => x軸の値
    - val_colname: 解析したい値の列名（必須） => y軸の値
    - winsize :  Windowサイズ（初期値:1Mb）
    - stepsize : Stepサイズ（初期値:0.2Mb）
    - chromsize : 染色体サイズ（必須）
    
    戻り値: 各Windowの中央値のリスト, 平均値のリスト
    """   
    
    win_position  = []  # 区間中央値用リスト
    win_snpindex = []  # 平均SNP-index用リスト

    n = 0 # 繰り返し数
    while True:
        #--- 区間のstartとend position ---
        start = stepsize * n 
        end   = start + winsize
        #--- 区間の中央値をリストに追加する ---
        p = (start + end) / 2
        win_position.append(p)
        #--- 区間内データを抽出 ---
        sub = df[(df[pos_colname] >= start) & (df[pos_colname] < end)]
        #--- SNP-indexの平均値を算出 ---
        i = sub[val_colname].mean()
        win_snpindex.append(i)
        #--- 繰り返し数を+1 ---
        n += 1
        #--- 全ての区間を調べた時、whileから出る ---
        if end > chromsize:
            break
    
    return win_position, win_snpindex


#=== メイン・プログラム ===
CHROM_SIZE = 23207287  # イネの第10染色体の長さ
WIN_SIZE = 1000000 # 1Mb
STEP_SIZE = 200000  # 0.2Mb

# 実データのSliding window解析
win_pos, win_snpidx = sliding_window(
    df2, 
    pos_colname='pos', 
    val_colname='snp_index', 
    winsize=WIN_SIZE, 
    stepsize=STEP_SIZE, 
    chromsize=CHROM_SIZE
)

# シミュレーションデータのSliding window解析
win_pos, win_simu95 = sliding_window(
    df2, 
    pos_colname='pos', 
    val_colname='simu95', 
    winsize=WIN_SIZE, 
    stepsize=STEP_SIZE, 
    chromsize=CHROM_SIZE
)

#--- グラフ作成: 各SNP-indexの散布図  ---
fig = plt.figure(figsize=[16,9])
plt.scatter(df2['pos'], df2['snp_index'], color='gray')      # 全データ
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)        # y軸ラベル

#--- Siliding Window解析の折れ線グラフ（重ね描き） ---
plt.plot(win_pos, win_snpidx, color='blue')  # 実データのSliding Window
plt.plot(win_pos, win_simu95, color='orange')  # シミュレーションデータのSliding Window   

20Mbを超えたあたりで、実データのSNP-indexのSliding windowがシミュレーションデータよりも高くなっています。  
そのゲノム領域は、帰無仮説「形質と無関係」が棄却される領域であり、「形質と関わりがある領域」と評価できます。  

さらに解析する場合、ゲノムポジションの区間情報（何番目の塩基から何番目の塩基まで）が必要です。  
その情報を得ましょう。

いま、Sliding window解析の各データは、次の変数にそれぞれリスト型で入っています。
- `win_pos`: 各windowのゲノムポジションの代表値（中央値）; windowサイズ=1Mbの中央値
- `win_snpidx`: 実データの平均SNP-index
- `win_simu95`: シミュレーションデータの平均SNP-index

In [ ]:
# Sliding window解析の各データを、リスト型からNumPy配列型に変換
# （2つのリストの値を比較するためにNumPy配列型に変換しています）

w_pos   = np.array(win_pos)        # 各windowのゲノムポジションの中央値
w_idx    = np.array(win_snpidx)   # 実データの平均SNP-index
w_simu = np.array(win_simu95) # 実データの平均SNP-index

# 実データとシミュレーションデータを比較し、条件に合うゲノムポジションを抽出
w_pos[w_idx > w_simu]

Windowのポジションが中央値であることを考慮すると、第10番染色体の21,000,000-22,600,000 bpが「形質と関わりがある領域」と判定できます。

実習では、この後の解析、突然変異形質「葉が淡緑色 (pale green)になる形質」の原因遺伝子を特定するまでをおこないませんが、上の解析で得られた領域内
の22,482,910bpの辺りに _Chlorophyllide a oxygenase_ (Os10t0567400) という遺伝子が存在しています。  
MutMapの論文(Abe et al., 2012)で、その遺伝子が、実際に「葉の色（緑色/淡緑色）」に関わっている証拠も得られており、突然変異形質の原因遺伝子と考えられています。 

---

ここ3回の実習で、大規模データの扱い方の基本から実際の解析（どういったことができるのか）までを体験してもらいました。
- ファイルの読み込み、書き出し
- 列同士の計算
- データのグラフ化
- Sliding window解析
- シミュレーションにより実データを評価

もし今後、研究や仕事で大量のデータを処理・解析する必要が生じた時に、今回学んだことが何らかの助けになれば幸いです。